In [1]:
import IntegralMethods as IM

import numpy as np

# Module test starts here.

In [2]:
# 1D weight test starts here. 
sample_grid = np.linspace(0, 10, 21, endpoint=True)
endpts = [4, 16]
print(sample_grid)
print(IM.get_1D_weight(sample_grid, endpts))

[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10. ]
[0.25 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.25]


# The above cell shows that the 1D weight calculation is performing properly. We now test the computation to get the full weight.

In [3]:
# test using a 4D weight, formatted as the output of  
test_weights = [[1,5,7,45],[1,6,8],[45,7],[3,5,7,8,2,65,78,89]]
W_F = IM.get_full_weight(test_weights)

# Goal: W_F[x, y,..., a] = test_weights[0][x]*test_weights[1][y]*...*test_weights[-1][a]

# for this test, we will just use a stacked for loop to make sure we are computing the right value. Generality is put aside for now.
for x in range(len(test_weights[0])):
    for y in range(len(test_weights[1])):
        for z in range(len(test_weights[2])):
            for a in range(len(test_weights[3])):
                if W_F[x, y, z, a] != test_weights[0][x]*test_weights[1][y]*test_weights[2][z]*test_weights[3][a]:
                    print("Failed")
                    break
print("Pass")

Pass


/src/notebooks/work/pysindy-nonlocal_interaction/integral/IntegralMethods.py:113: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  W_F = W_F[index] * np.array(weights[w])


# Reshape Demo

Some interesting things I've found:

    Out here we use A.reshape to reshape an array, and the transformation from our original matrix to 2D data matrix X is indeed simply A.reshape.

In [4]:
# Methods to create a testing input X

from typing import List
from sklearn.base import TransformerMixin
HANDLED_FUNCTIONS = {}

class AxesArray(np.lib.mixins.NDArrayOperatorsMixin, np.ndarray):
    """A numpy-like array that keeps track of the meaning of its axes.

    Paramters:
        input_array (array-like): the data to create the array.
        axes (dict): A dictionary of axis labels to shape indices.
            Allowed keys:
                - ax_time: int
                - ax_coord: int
                - ax_sample: int
                - ax_spatial: List[int]

    Raises:
        AxesWarning if axes does not match shape of input_array
    """

    def __new__(cls, input_array, axes):
        obj = np.asarray(input_array).view(cls)
        defaults = {
            "ax_time": None,
            "ax_coord": None,
            "ax_sample": None,
            "ax_spatial": [],
        }
        if axes is None:
            return obj
        obj.__dict__.update({**defaults, **axes})
        return obj

    def __array_finalize__(self, obj) -> None:
        if obj is None:
            return
        self.ax_time = getattr(obj, "ax_time", None)
        self.ax_coord = getattr(obj, "ax_coord", None)
        self.ax_sample = getattr(obj, "ax_sample", None)
        self.ax_spatial = getattr(obj, "ax_spatial", [])

    @property
    def n_spatial(self):
        return tuple(self.shape[ax] for ax in self.ax_spatial)

    @property
    def n_time(self):
        return self.shape[self.ax_time] if self.ax_time is not None else 1

    @property
    def n_sample(self):
        return self.shape[self.ax_sample] if self.ax_sample is not None else 1

    @property
    def n_coord(self):
        return self.shape[self.ax_coord] if self.ax_coord is not None else 1

    def __array_ufunc__(
        self, ufunc, method, *inputs, out=None, **kwargs
    ):  # this method is called whenever you use a ufunc
        args = []
        for input_ in inputs:
            if isinstance(input_, AxesArray):
                args.append(input_.view(np.ndarray))
            else:
                args.append(input_)

        outputs = out
        if outputs:
            out_args = []
            for output in outputs:
                if isinstance(output, AxesArray):
                    out_args.append(output.view(np.ndarray))
                else:
                    out_args.append(output)
            kwargs["out"] = tuple(out_args)
        else:
            outputs = (None,) * ufunc.nout
        results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
        if results is NotImplemented:
            return NotImplemented
        if method == "at":
            return
        if ufunc.nout == 1:
            results = (results,)
        results = tuple(
            (AxesArray(np.asarray(result), self.__dict__) if output is None else output)
            for result, output in zip(results, outputs)
        )
        return results[0] if len(results) == 1 else results

    def __array_function__(self, func, types, args, kwargs):
        if func not in HANDLED_FUNCTIONS:
            arr = super(AxesArray, self).__array_function__(func, types, args, kwargs)
            if isinstance(arr, np.ndarray):
                return AxesArray(arr, axes=self.__dict__)
            elif arr is not None:
                return arr
            return
        if not all(issubclass(t, AxesArray) for t in types):
            return NotImplemented
        return HANDLED_FUNCTIONS[func](*args, **kwargs)
    
# This is how we created X from the original list of stuff, 
def concat_sample_axis(x_list: List[AxesArray]):
    """Concatenate all trajectories and axes used to create samples."""
    new_arrs = []
    for x in x_list:
        sample_axes = (
            x.ax_spatial
            + ([x.ax_time] if x.ax_time is not None else [])
            + ([x.ax_sample] if x.ax_sample is not None else [])
        )
        
#         print(sample_axes)
        
        new_axes = {"ax_sample": 0, "ax_coord": 1}
        n_samples = np.prod([x.shape[ax] for ax in sample_axes])
        
#         print(n_samples)
        
#         the new 2D data matrix is literally created with a reshape
#         print(x.reshape((n_samples, x.shape[x.ax_coord])))
        arr = AxesArray(x.reshape((n_samples, x.shape[x.ax_coord])), new_axes)
#         Actually, this is problematic. We only did a reshape without doing any filtering and stuff 
#         so we cannot guarantee each column is indeed a feature
        
#         and each 2D data matrix (for their corresponding trajectory) is put into a list. 
        new_arrs.append(arr)
    return np.concatenate(new_arrs, axis=new_arrs[0].ax_sample)

In [5]:
# space_1, space_2, t, coord(feature)
# We carry the assumption that the first d-2 axis are spatial, second last is time, and last is coords. 
A = np.random.rand(12, 13, 7, 2)
axes = {"ax_spatial": [0, 1], "ax_time": 2, "ax_coord": 3}
A_ = AxesArray(A, axes)

# need brackets around A_ as input is list of trajectories
A_2 = concat_sample_axis([A_])

# indeed, an reshape retrieves the original matrix.
np.linalg.norm(A_2.reshape(A.shape)-A)

0.0

# We now test the part where we filter data points in a subdomain out of X.

### We carry the assumption that the first d-2 axis are spatial, second last is time, and last is coords. 

In [6]:
from scipy.io import loadmat

# Construct a sample input for our program.
data = loadmat('pysindy-master/examples/data/burgers.mat')
time = np.ravel(data['t'])
x = np.ravel(data['x'])
X, T = np.meshgrid(x, time)

axes = {"ax_spatial": [0], "ax_time": 1, "ax_coord": 2}
A = AxesArray(np.asarray([X, T]).T, axes)

# Here we only care about the size of the grid, which will match the size of A without the feature axis.
# The last axis won't matter so we just put a 1 there
# spatiotemporal_grid is a parameter of our program
spatiotemporal_grid = np.zeros(list(np.shape(np.asarray([X, T]).T)[0:-1]) + [1])

# A_ is a parameter of our program
A_ = concat_sample_axis([A])

Data = IM.retrieve_data_mat(spatiotemporal_grid, A_)

print(np.linalg.norm(Data-A))

0.0


### With Data matrix retrieved, we now filter it.

In [7]:
# We want the first feature
j = 0
# We want the 50-th time point
t_ind = 50
# The subdomain is bounded by the 10-th and 20-th point. We only have 1D but if we have more dimensions, 
# x_bound is expected to be a list of list, with bounds of each spatial dimensions in order.
x_bound = [[10, 20]]

# +1 to include the last point
Expected = np.asarray([X, T]).T[x_bound[0][0]:x_bound[0][1], 50, 0]

obtained = IM.filterX(Data, j, x_bound, t_ind)

print(np.linalg.norm(Expected - obtained))

0.0


# Test 1D integral calculation against trapezoidal rule.

In [8]:
# Generate Data
from sklearn.metrics import mean_squared_error
from scipy.io import loadmat

# Construct a sample input for our program.
data = loadmat('pysindy-master/examples/data/burgers.mat')
time = np.ravel(data['t'])
x = np.ravel(data['x'])
X, T = np.meshgrid(x, time)

u = np.real(data['usol'])
rmse = mean_squared_error(u, np.zeros(u.shape), squared=False)
u = u + np.random.normal(0, rmse / 5.0, u.shape)
u = np.reshape(u, (len(x), len(time), 1))
axes = {"ax_spatial": [0], "ax_time": 1, "ax_coord": 2}
U = AxesArray(u, axes)

spatiotemporal_grid = np.asarray([X, T]).T

In [9]:
# Setup sample parameters
j = 0
t = 10
endpts = [[0, 90]]

In [11]:
# Construct reference value
ref = np.trapz(U[endpts[0][0]:endpts[0][1], t, j], x=spatiotemporal_grid[endpts[0][0]:endpts[0][1], t, 0])

# Apply our program
obtained = IM.compute_integral(U, spatiotemporal_grid, t, j, endpts)
    
print(abs(ref-obtained))

/src/notebooks/work/pysindy-nonlocal_interaction/integral/IntegralMethods.py:63: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  this_dim = spatiotemporal_grid[index]


TypeError: 'int' object is not subscriptable

# Test with 2D reaction diffusion equation

In [ ]:
from numpy.fft import fft2, ifft2
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Define the reaction-diffusion PDE in the Fourier (kx, ky) space
def reaction_diffusion(t, uvt, K22, d1, d2, beta, n, N):
    ut = np.reshape(uvt[:N], (n, n))
    vt = np.reshape(uvt[N : 2 * N], (n, n))
    u = np.real(ifft2(ut))
    v = np.real(ifft2(vt))
    u3 = u ** 3
    v3 = v ** 3
    u2v = (u ** 2) * v
    uv2 = u * (v ** 2)
    utrhs = np.reshape((fft2(u - u3 - uv2 + beta * u2v + beta * v3)), (N, 1))
    vtrhs = np.reshape((fft2(v - u2v - v3 - beta * u3 - beta * uv2)), (N, 1))
    uvt_reshaped = np.reshape(uvt, (len(uvt), 1))
    uvt_updated = np.squeeze(
        np.vstack(
            (-d1 * K22 * uvt_reshaped[:N] + utrhs, 
             -d2 * K22 * uvt_reshaped[N:] + vtrhs)
        )
    )
    return uvt_updated


# Generate the data
t = np.linspace(0, 10, int(10 / 0.1))
d1 = 0.1
d2 = 0.1
beta = 1.0
L = 20  # Domain size in X and Y directions
n = 64  # Number of spatial points in each direction
N = n * n
x_uniform = np.linspace(-L / 2, L / 2, n + 1)
x = x_uniform[:n]
y = x_uniform[:n]
n2 = int(n / 2)
# Define Fourier wavevectors (kx, ky)
kx = (2 * np.pi / L) * np.hstack((np.linspace(0, n2 - 1, n2), 
                                  np.linspace(-n2, -1, n2)))
ky = kx
# Get 2D meshes in (x, y) and (kx, ky)
X, Y = np.meshgrid(x, y)
KX, KY = np.meshgrid(kx, ky)
K2 = KX ** 2 + KY ** 2
K22 = np.reshape(K2, (N, 1))

m = 1  # number of spirals

# define our solution vectors
u = np.zeros((len(x), len(y), len(t)))
v = np.zeros((len(x), len(y), len(t)))

# Initial conditions
u[:, :, 0] = np.tanh(np.sqrt(X ** 2 + Y ** 2)) * np.cos(
    m * np.angle(X + 1j * Y) - (np.sqrt(X ** 2 + Y ** 2))
)
v[:, :, 0] = np.tanh(np.sqrt(X ** 2 + Y ** 2)) * np.sin(
    m * np.angle(X + 1j * Y) - (np.sqrt(X ** 2 + Y ** 2))
)

# uvt is the solution vector in Fourier space, so below
# we are initializing the 2D FFT of the initial condition, uvt0
uvt0 = np.squeeze(
    np.hstack(
        (np.reshape(fft2(u[:, :, 0]), (1, N)), 
         np.reshape(fft2(v[:, :, 0]), (1, N)))
    )
)

# Solve the PDE in the Fourier space, where it reduces to system of ODEs
uvsol = solve_ivp(
    reaction_diffusion, 
    (t[0], t[-1]), 
    y0=uvt0, 
    t_eval=t, 
    args=(K22, d1, d2, beta, n, N)
)
uvsol = uvsol.y

# Reshape things and ifft back into (x, y, t) space from (kx, ky, t) space
for j in range(len(t)):
    ut = np.reshape(uvsol[:N, j], (n, n))
    vt = np.reshape(uvsol[N:, j], (n, n))
    u[:, :, j] = np.real(ifft2(ut))
    v[:, :, j] = np.real(ifft2(vt))

dt = t[1] - t[0]
dx = x[1] - x[0]
dy = y[1] - y[0]

u_sol = u
v_sol = v

U = np.zeros((n, n, len(t), 2))
U[:, :, :, 0] = u_sol
U[:, :, :, 1] = v_sol
X, Y, T = np.meshgrid(x, y, t, indexing='ij')
XYT = np.transpose([X, Y, T], [1, 2, 3, 0])

### Data matrix is U, spatiotemporal_grid is XYT

In [ ]:
j = 0
t = 10
endpts = [[0, 10], [0, 10]]

In [ ]:
length = endpts[-1][1]-endpts[-1][0]
X = []
for i in range(length):
    res = np.trapz(U[endpts[0][0]:endpts[0][1], i, t, j], x=XYT[endpts[0][0]:endpts[0][1], i, t, 0])
    
    X.append(res)
    
ref = np.trapz(X, x=XYT[0, endpts[1][0]:endpts[1][1], t, 1])
print(ref)

## Validate Ref Value with another approach to 2D trapz.

In [ ]:
xarr = XYT[0:10, 0, 0, 0]
yarr = XYT[0, 0:10, 0, 1]
ref2 = 0
# area = np.zeros((len(xarr)-1)*(len(yarr)-1))
for i in np.arange(len(xarr)-1):
    for k in np.arange(len(yarr)-1):
        area = (xarr[i+1]-xarr[i])*(yarr[k+1]-yarr[k])
        weight = (U[i, k, t, j]+U[i+1, k, t, j]+U[i, k+1, t, j]+U[i+1, k+1, t, j])/4
        ref2 += area*weight
print(abs(ref-ref2))

## Compare result with our program.

In [ ]:
obtained = IM.compute_integral(U, XYT, t, j, endpts)
print(abs(ref-obtained))